In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import srsly
import pandas as pd

from glob import glob
from tqdm.auto import tqdm

In [ ]:
public = pd.read_csv("/workspace/resources/data/dump-20230630/set_de_datos_con_perspectiva_de_genero.csv")

public.dropna(subset=["NRO_REGISTRO", "TOMO", "FECHA_RESOLUCION"], inplace=True)
public.rename(columns={c: c.lower() for c in public.columns}, inplace=True)

public["tomo"] = public["tomo"].astype(int)


def fix_date(text: str):
    date = text.split("_")
    # print(date)
    if len(date) != 3:
        return

    for i, num in enumerate(date):
        num = int(num)
        date[i] = f"{num:02.0f}"
    try:
        return pd.to_datetime("/".join(date))
    except:
        return


public["fecha_resolucion"] = [fix_date(date) for date in tqdm(public["fecha_resolucion"])]

public

In [ ]:
from aymurai.datasets.ar_juz_pcyf_10.annotations import (
    ArgentinaJuzgadoPCyF10LabelStudioAnnotations,
)

annotations = ArgentinaJuzgadoPCyF10LabelStudioAnnotations("/workspace/resources/annotations")

In [ ]:
from aymurai.utils.misc import get_element


def text_normalize(text: str):
    text = text.replace("\/", "/")
    return text


metadata = pd.DataFrame(
    [
        x["metadata"]
        | x["data"]
        | {
            "original_path": x["path"],
            # "conll": get_element(x, levels=["annotations", "conll"]),
        }
        for x in annotations.data
    ]
)
metadata.dropna(subset=["tomo"], inplace=True)
metadata = metadata[metadata["tomo"].str.isnumeric()]
metadata["tomo"] = metadata["tomo"].astype(int)
metadata["original_path"] = metadata["original_path"].apply(text_normalize)
metadata["doc.text"] = metadata["doc.text"].apply(text_normalize)

# metadata['tomo'] = metadata['tomo'].astype(int)
metadata.drop_duplicates(["tomo", "nro_registro"], inplace=True)
metadata

In [ ]:
# Inner join
x = pd.merge(public, metadata, on=["nro_registro", "tomo"], how="inner")
x.drop_duplicates(["tomo", "nro_registro"], inplace=True)
x.info()

In [ ]:
import os
import gdown
import subprocess
from hashlib import blake2b
from tqdm.auto import tqdm
from joblib import Parallel, delayed

tqdm.pandas()

outdir = "/resources/data/documents"
os.makedirs(outdir, exist_ok=True)


def get_file(url: str):
    if not isinstance(url, str):
        return

    fname = f"{outdir}/{blake2b(url.encode(), digest_size=15).hexdigest()}"

    if os.path.exists(fname):
        return fname

    # # algunos archivos requieren acceso, otros estan corruptos
    # cmd = f"gdown --fuzzy -q --continue -O {fname} {url}"
    try:
        pass
        # fname = gdown.download(url, fname, quiet=False, fuanon_tfidfzzy=True, resume=True)
        # output = subprocess.getoutput(cmd)
    except:
        return

    # if "Access denied" in output:
    # return

    return fname


parallel = Parallel(n_jobs=50, backend="threading")
get_file_ = delayed(get_file)
path = parallel(get_file_(url) for url in tqdm(x["link"]))

In [ ]:
x["anonymized_path"] = path

In [ ]:
x.sample(5)

In [ ]:
x.info()

In [ ]:
# 2021 - 2022, con versión anonimizada
mask = (
    (x["fecha_resolucion"].dt.to_period("Y") >= "2021")
    & (x["fecha_resolucion"].dt.to_period("Y") <= "2022")
    & (x["anonymized_path"].notna())
)
x = x.loc[mask]
x

In [ ]:
x.groupby(x["fecha_resolucion"].dt.to_period("Y"))["anonymized_path"].count()

In [ ]:
import textract

IA2_START_DATE = pd.to_datetime("01/01/2021")


def load_doc(path: str):
    doc = textract.process(path, extension="odt")
    return doc.decode("utf-8")


def ia2(row):
    path = row["anonymized_path"]
    date = row["fecha_resolucion"]
    if date < IA2_START_DATE:
        return False
    try:
        doc = load_doc(path)
    except Exception as e:
        return False
    return "XX" not in doc

In [ ]:
x["ia2"] = [ia2(row) for i, row in tqdm(x.iterrows())]

In [ ]:
x.query("ia2")["fecha_resolucion"].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
# Filtro de resoluciones con etiquetas de ia2
mask &= x["ia2"]
x = x.loc[mask]
x.info()

In [ ]:
x["original_path"] = (
    x["original_path"]
    .str.replace("/resources/", "/resources/data/")
    .str.replace("/RESOLUCIONES DEL JUZGADO - DOCS/", "/RESOLUCIONES DEL JUZGADO/")
)

In [ ]:
import os
import re
from aymurai.utils import alignment
from collections import Counter


OUTPUT_DIR = "/resources/data/restricted/anonymization"
os.makedirs(OUTPUT_DIR, exist_ok=True)

labels = Counter()

x.reset_index(drop=True, inplace=True)

for i, row in tqdm(x.iterrows(), total=len(x)):
    original_path = row["original_path"]
    anonymized_path = row["anonymized_path"]

    mapping = alignment.align_docs(
        original_path,
        anonymized_path,
        columns=("original", "anonymized"),
        target_preprocess=alignment.ia2.ia2_text_preprocess,
    )

    # alignment_score
    diff = mapping["original"] != mapping["anonymized"]
    alignment_score = (~diff).mean()
    x.loc[i, "alignment_score"] = alignment_score

    # export alignment
    filename = f"{OUTPUT_DIR}/alignment/{row['tomo']}_{row['nro_registro']}.csv"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    x.loc[i, "alignment_path"] = filename
    mapping.to_csv(filename, index=False)

    diff_text = "".join(mapping.loc[diff, "anonymized"])
    labels_ = re.findall(r"<\w+>", diff_text)
    labels_ = [alignment.ia2.normalize(label) for label in labels_]
    labels += Counter(labels_)

In [ ]:
columns = [
    "nro_registro",
    "tomo",
    "fecha_resolucion",
    "n_expte_eje",
    "original_path",
    "anonymized_path",
    "doc.text",
    "alignment_path",
    "alignment_score",
]

x[columns].head()

In [ ]:
x[columns].sort_values("alignment_score", ascending=False)

In [ ]:
x["alignment_score"].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99])

In [ ]:
x.query("alignment_score < 0.7")["alignment_path"].values

In [ ]:
import re
import os
import functools
from collections import Counter

from aymurai.utils import alignment
from aymurai.text.extraction import extract_document


IA2_LABELS = pd.read_csv("/resources/data/restricted/anonymization/ia2_labels_manual.csv")
IA2_LABELS_MAPPING = {x["label"]: x["mapping"] for _, x in IA2_LABELS.iterrows()}

for i, row in tqdm(x.iterrows(), total=len(x)):
    mapping = pd.read_csv(row["alignment_path"])
    original = extract_document(row["original_path"])

    mapping = alignment.core.add_empty_lines_between_paragraphs(original, mapping)

    diff = mapping["original"] != mapping["anonymized"]
    diff_text = "".join(mapping.loc[diff, "anonymized"].fillna(""))
    labels = set(re.findall(r"<\w+>", diff_text))
    labels = [alignment.ia2.normalize(label) for label in labels]

    _norm_ia2_label = functools.partial(alignment.ia2.norm_ia2_label, labels=labels)

    mask = mapping["original"] != mapping["anonymized"]
    mapping.loc[mask, "label"] = mapping.loc[mask, "anonymized"]
    mapping["label"] = mapping["label"].apply(_norm_ia2_label)
    mapping["label"] = mapping["label"].apply(
        lambda x: IA2_LABELS_MAPPING.get(x.strip()) if isinstance(x, str) else None
    )
    mapping["label"] = alignment.ia2.label_to_conll_format(mapping["label"])
    mapping["label"] = mapping["label"].fillna("O")

    if len(mapping.loc[mapping["label"].str.contains("NUM_DOMINIO")]) > 1:
        display(mapping.loc[mapping["label"].str.contains("NUM_DOMINIO")])

    # patch blank lines in labels
    mask = mapping["original"] == ""
    mapping.loc[mask] = ""

    # export mapping
    filename = f"{OUTPUT_DIR}/annotation/{row['tomo']}_{row['nro_registro']}.csv"
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    x.loc[i, "annotation_path"] = filename
    mapping.to_csv(filename, index=False)

    mapping.loc[:, row.index] = row.values

In [ ]:
ANNOT_DIR = "/resources/data/restricted/anonymization/annotation/"

TO_REMOVE = [
    "38_3531.csv",
    "42_3831.csv",
    "36_3404.0.csv",
    "36_3406.0.csv",
    "36_3416.0.csv",
    "37_3449.0.csv",
    "38_3519.0.csv",
    "40_3724.0.csv",
    "41_3772.0.csv",
    "41_3793.0.csv",
    "41_3800.0.csv",
    "42_3821.1.csv",
    "43_3918.0.csv",
    "44_4044.0.csv",
    "44_4049.0.csv",
    "47_4234.0.csv",
    "47_4235.0.csv",
    "47_4319.0.csv",
    "50_4594.0.csv",
    "40_3721.csv",  # el problema es la nota al pie de página
    "39_3613.csv",  # el problema es la nota al pie de página
    "40_3674.csv",  # el problema es la nota al pie de página
    "35_3328.csv",  # el problema es la nota al pie de página
]

In [ ]:
data = [csv for csv in os.listdir(ANNOT_DIR) if csv not in TO_REMOVE]
len(data)

In [ ]:
sorted(data)

In [ ]:
from aymurai.utils.display.pandas import pandas_context
from rich.pretty import pprint
import functools
import random
import re


options = {
    "display.max_rows": 500,
    "display.max_columns": 500,
    "display.width": 0,
    "display.max_rows": None,
}

example = random.choice(data)
mapping = pd.read_csv(os.path.join(ANNOT_DIR, example))
mapping.fillna("", inplace=True)
pprint(example)

# with pandas_context(**options):
# display(mapping.head())

In [ ]:
mask = (mapping["anonymized"].map(lambda x: True if re.match(r"(?:<\w+>\|<?\w+>?\|)+", x) else False)) & (
    mapping["label"] == "O"
)


mapping.loc[mask]

In [ ]:
full_data = pd.concat(
    [
        pd.concat(
            [
                pd.read_csv(os.path.join(ANNOT_DIR, csv)),
                pd.Series(
                    [csv] * len(pd.read_csv(os.path.join(ANNOT_DIR, csv))),
                    name="filename",
                ),
            ],
            axis=1,
        )
        for csv in data
    ],
    ignore_index=True,
)
full_data

In [ ]:
full_data_nan = full_data.dropna().copy()
full_data_nan.info()

In [ ]:
mask = (full_data_nan["anonymized"].map(lambda x: True if re.match(r"(?:<\w+>\|<?\w+>?\|)+", x) else False)) & (
    full_data_nan["label"] == "O"
)

# with pandas_context(**options):
#     display(full_data.loc[mask])

In [ ]:
len(full_data_nan.loc[mask]) / len(full_data_nan) * 100

In [ ]:
full_data_nan.loc[mask, "label"] = None

In [ ]:
full_data_nan["label"].isna().sum()

In [ ]:
full_data.loc[full_data_nan.loc[mask].index, "label"] = None
full_data.info()

In [ ]:
full_data["label"].value_counts()

In [ ]:
full_data["label"].dropna().map(lambda x: re.sub(r"^[BI]-", "", x)).value_counts()

In [ ]:
full_data.dropna().loc[full_data.dropna()["label"].str.contains("NUM_DOMINIO")]

In [ ]:
nums = full_data.dropna().loc[full_data["label"].dropna().str.contains("NUM_$"), "anonymized"].unique()
nums

In [ ]:
mask_num = full_data.dropna()["label"].str.contains("NUM_$")

In [ ]:
mask_expte = mask_num & (full_data.dropna()["anonymized"].str.contains(r"EXPEDIENTE|CAUSA", regex=True))

In [ ]:
full_data.dropna().loc[mask_expte]

In [ ]:
expte_idx = full_data.dropna().loc[mask_expte].index
full_data.loc[expte_idx, "label"] = full_data.loc[expte_idx, "label"].str.replace("NUM_", "NUM_EXPEDIENTE")

In [ ]:
mask_dni = mask_num & (full_data.dropna()["anonymized"].str.contains("DNI"))

In [ ]:
full_data.dropna().loc[mask_dni]

In [ ]:
dni_idx = full_data.dropna().loc[mask_dni].index
full_data.loc[dni_idx, "label"] = full_data.loc[dni_idx, "label"].str.replace("NUM_", "DNI")

In [ ]:
mask_cuij = mask_num & (full_data.dropna()["anonymized"].str.contains("CUIJ"))

In [ ]:
full_data.dropna().loc[mask_cuij]

In [ ]:
cuij_idx = full_data.dropna().loc[mask_cuij].index
full_data.loc[cuij_idx, "label"] = full_data.loc[cuij_idx, "label"].str.replace("NUM_", "CUIJ")

In [ ]:
mask_num_ = mask_num & (full_data.dropna()["label"].str.endswith("NUM_"))

In [ ]:
num_idx = full_data.dropna().loc[mask_num_].index
full_data.loc[num_idx, "label"] = full_data.loc[num_idx, "label"].str.replace("NUM_", "NUM")

In [ ]:
# FIXME este mapeo debería hacerser automáticamente a partir de IA2_LABELS_MAPPING
# full_data["label"] = full_data["label"].str.replace("NUM_ACTUACION", "NUM_EXPEDIENTE")
full_data["label"] = full_data["label"].str.replace("DOMINIO_PATENTE", "PATENTE_DOMINIO")
full_data["label"] = full_data["label"].str.replace("NUM_DOMINIO", "PATENTE_DOMINIO")

In [ ]:
full_data["label"] = full_data["label"].str.replace(r"[BI]-(HECHO)", "FECHA", regex=True)

In [ ]:
full_data["label"].dropna().map(lambda x: re.sub(r"^[BI]-", "", x)).value_counts()

In [ ]:
full_data.query("label == 'I-NUM'")

In [ ]:
# TODO anular labels para conectores tipo y/o
full_data.query("(original == 'y' or original == 'Y') and (label != 'O')")

In [ ]:
# TODO anular labels para conectores tipo y/o
full_data.query("(original == 'o' or original == 'O') and (label != 'O')")

In [ ]:
# TODO anular labels para conectores tipo y/o
full_data.query("(original == 'y/o' or original == 'Y/O') and (label != 'O')")

In [ ]:
full_data.loc[(full_data["label"].isna()) & (full_data["original"].notna())]

In [ ]:
pd.set_option("display.max_rows", 100)

In [ ]:
full_data.loc[(full_data["label"].isna()) & (full_data["original"].notna()), "filename"].unique()

In [ ]:
len(full_data.loc[(full_data["label"].isna()) & (full_data["original"].notna()), "filename"].unique())

In [ ]:
len(
    full_data.loc[
        ((full_data["original"].notnull()) & (full_data["anonymized"].isnull()))
        | (full_data["label"].isnull()) & (full_data["original"].notnull()),
        "filename",
    ].unique()
)

In [ ]:
import numpy as np
from more_itertools import consecutive_groups


def remove_ambiguous_labels(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    to_annul = (
        df["anonymized"].map(
            lambda x: True if (isinstance(x, str) and re.match(r"(?:<\w+>\|<?\w+>?\|)+", x)) or (x is None) else False
        )
    ) & (df["label"] == "O")

    df.loc[to_annul, "label"] = None

    to_remove = ((df["original"].notnull()) & (df["label"].isnull())) | (
        (df["original"].notnull()) & (df["anonymized"].isnull())
    )

    to_remove_idx = [list(item) for item in consecutive_groups(df.loc[to_remove].index)]

    null_row_idx = df.loc[(df["original"].isnull()) & (df["label"].isnull())].index

    for idx in to_remove_idx:
        min_idx = min(idx)
        diff = null_row_idx - min_idx
        neg_diff = diff[diff < 0]
        argmax_neg_diff = np.argmax(neg_diff)

        paragraph_start = null_row_idx[argmax_neg_diff]
        paragraph_end = null_row_idx[argmax_neg_diff + 1] if min_idx < max(null_row_idx) else len(df)

        df.drop(range(paragraph_start, paragraph_end), inplace=True, errors="ignore")

    return df


def normalize_num_label(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    mask_num = df["label"].str.contains("NUM_$", na=False)

    # NUM_EXPEDIENTE
    mask_expte = mask_num & (df["anonymized"].str.contains(r"EXPEDIENTE|CAUSA", na=False, regex=True))
    expte_idx = df.loc[mask_expte].index
    df.loc[expte_idx, "label"] = df.loc[expte_idx, "label"].str.replace("NUM_", "NUM_EXPEDIENTE")

    # DNI
    mask_dni = mask_num & (df["anonymized"].str.contains("DNI", na=False))
    dni_idx = df.loc[mask_dni].index
    df.loc[dni_idx, "label"] = df.loc[dni_idx, "label"].str.replace("NUM_", "DNI")

    # CUIJ
    mask_cuij = mask_num & (df["anonymized"].str.contains("CUIJ", na=False))
    cuij_idx = df.loc[mask_cuij].index
    df.loc[cuij_idx, "label"] = df.loc[cuij_idx, "label"].str.replace("NUM_", "CUIJ")

    # NUM
    mask_num_ = mask_num & (df["label"].str.endswith("NUM_", na=False))
    num_idx = df.loc[mask_num_].index
    df.loc[num_idx, "label"] = df.loc[num_idx, "label"].str.replace("NUM_", "NUM")

    return df


def normalize_other_labels(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # FIXME este mapeo debería hacerse automáticamente a partir de IA2_LABELS_MAPPING
    # PATENTE_DOMINIO
    df["label"] = df["label"].str.replace("DOMINIO_PATENTE", "PATENTE_DOMINIO")
    df["label"] = df["label"].str.replace("NUM_DOMINIO", "PATENTE_DOMINIO")

    # FECHA
    df["label"] = df["label"].str.replace(r"[BI]-(HECHO)", "FECHA", regex=True)

    return df

In [ ]:
OUTPUT_DIR = "/resources/data/restricted/anonymization/to-review"
os.makedirs(OUTPUT_DIR, exist_ok=True)

modified = []

for csv in tqdm(data):
    df = pd.read_csv(os.path.join(ANNOT_DIR, csv))
    original_len = len(df)

    # df = remove_ambiguous_labels(df)
    new_len = len(df)

    df = normalize_num_label(df)
    df = normalize_other_labels(df)

    filename = os.path.join(OUTPUT_DIR, csv)
    df.to_csv(filename, index=False)

    # if new_len != original_len:
    modified.append(csv)

len(modified)

In [ ]:
revised_annots_paths = [os.path.join(OUTPUT_DIR, file) for file in os.listdir(OUTPUT_DIR) if file.endswith(".csv")]
print("len", len(revised_annots_paths))
revised_annots_paths[:5]

In [ ]:
revised_annots_paths = sorted(list(set(revised_annots_paths)))
print("len", len(revised_annots_paths))
revised_annots_paths[:5]

In [ ]:
OUTPUT_DIR

In [ ]:
# output files list
with open(f"{OUTPUT_DIR}/files.list", "w") as file:
    file.write("\n".join(list(map(os.path.basename, revised_annots_paths))))

In [ ]:
with open(f"{OUTPUT_DIR}/files.list") as file:
    revised_annots_paths_loaded = file.readlines()
    revised_annots_paths_loaded = [f"{OUTPUT_DIR}/{f.strip()}" for f in revised_annots_paths_loaded]
revised_annots_paths_loaded

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(revised_annots_paths_loaded, test_size=0.2, random_state=42)
dev, test = train_test_split(test, test_size=0.5, random_state=42)

In [ ]:
len(train), len(dev), len(test)

In [ ]:
train[0], dev[0], test[0]

In [ ]:
assert len(train) + len(dev) + len(test) == len(revised_annots_paths)

In [ ]:
assert (
    (len(set(train).intersection(set(dev))) == 0)
    & (len(set(train).intersection(set(test))) == 0)
    & (len(set(dev).intersection(set(test))) == 0)
)

In [ ]:
OUTPUT_DIR = "/resources/data/restricted/anonymization"


with open(f"{OUTPUT_DIR}/train.list", "w") as file:
    file.write("\n".join(list(map(os.path.basename, train))))

with open(f"{OUTPUT_DIR}/dev.list", "w") as file:
    file.write("\n".join(list(map(os.path.basename, dev))))

with open(f"{OUTPUT_DIR}/test.list", "w") as file:
    file.write("\n".join(list(map(os.path.basename, test))))

In [ ]:
df_train = pd.concat([pd.read_csv(doc)[["original", "label"]] for doc in train], ignore_index=True)
train_conll = df_train.to_csv(sep=" ", header=False, index=False)

df_dev = pd.concat([pd.read_csv(doc)[["original", "label"]] for doc in dev], ignore_index=True)
dev_conll = df_dev.to_csv(sep=" ", header=False, index=False)

df_test = pd.concat([pd.read_csv(doc)[["original", "label"]] for doc in test], ignore_index=True)
test_conll = df_test.to_csv(sep=" ", header=False, index=False)

In [ ]:
# NOTE recordar shuffle al momento de entrenar
# Random shuffle - paragraphs
# Split by paragraph
# train_conll = [paragraph for paragraph in train_conll.split("\n \n") if paragraph != ""]

# Shuffle
# random.seed(42)
# random.shuffle(train_conll)

# Rejoin annotations
# train_conll = "\n\n".join(train_conll)

# export to conll format

In [ ]:
with open(os.path.join(OUTPUT_DIR, "train.txt"), "w") as file:
    file.write(train_conll)

with open(os.path.join(OUTPUT_DIR, "dev.txt"), "w") as file:
    file.write(dev_conll)

with open(os.path.join(OUTPUT_DIR, "test.txt"), "w") as file:
    file.write(test_conll)

# export to huggingface format

In [ ]:
train = pd.read_csv(
    f"{OUTPUT_DIR}/train.txt",
    sep=" ",
    names=["token", "label"],
    skip_blank_lines=False,
)

val = pd.read_csv(
    f"{OUTPUT_DIR}/dev.txt",
    sep=" ",
    names=["token", "label"],
    skip_blank_lines=False,
)

test = pd.read_csv(
    f"{OUTPUT_DIR}/test.txt",
    sep=" ",
    names=["token", "label"],
    skip_blank_lines=False,
)

cat_count = dict(train["label"].value_counts())
categories = {v: k for k, v in enumerate(train["label"].unique()) if not pd.isna(v)}
label2code = {k: i for i, (k, v) in enumerate(categories.items())}
code2label = {v: k for k, v in label2code.items()}

print("train:", len(train))
print("val:", len(val))
print("test:", len(test))
print("nlabels:", len(code2label))

In [ ]:
import numpy as np
from more_itertools import unzip
from tqdm.auto import tqdm
from more_itertools import split_at
from functools import cache


def get_hg_format(df):
    df = df.copy()

    df["tags"] = df["label"].map(lambda x: label2code.get(x, label2code["O"]))
    items = []
    indices, rows = unzip(df.iterrows())
    tuples = map(lambda row: (row["token"], row["tags"]), rows)
    tuples = split_at(tuples, lambda x: pd.isna(x[0]))

    for i, paragraph in enumerate(tuples):
        df = pd.DataFrame(paragraph)
        if not len(df):
            continue
        tokens = df[0].values
        labels = df[1].values.astype(int)
        nlabels = (len(labels[labels > 0]),)

        if any(np.isnan(labels)):
            continue

        if len(tokens) != len(labels):
            print("mismatch size")
            continue

        items.append(
            {
                "n_labels": nlabels,
                "tokens": list(tokens),
                "tags": labels,
            }
        )

    return items

In [ ]:
from tqdm import tqdm
from datasets import DatasetDict, Dataset

dataset = DatasetDict(
    {
        "train": Dataset.from_list(get_hg_format(train)),
        "validation": Dataset.from_list(get_hg_format(val)),
        "test": Dataset.from_list(get_hg_format(test)),
    }
)
dataset

### drop duplicates

In [ ]:
df_train = dataset["train"].to_pandas()
df_dev = dataset["validation"].to_pandas()
df_test = dataset["test"].to_pandas()

# apply hash to fast compare dupplicated
df_train["hash"] = df_train["tokens"].str.join(" ").apply(hash)
df_dev["hash"] = df_dev["tokens"].str.join(" ").apply(hash)
df_test["hash"] = df_test["tokens"].str.join(" ").apply(hash)

# drop duplicates
df_train.drop_duplicates(subset="hash", inplace=True)
df_dev.drop_duplicates(subset="hash", inplace=True)
df_test.drop_duplicates(subset="hash", inplace=True)

# get train hashes
train_hash = set(df_train["hash"])
dev_hash = set(df_dev["hash"])

In [ ]:
options["display.max_colwidth"] = 0

with pandas_context(**options):
    aux = df_test.query("hash in @train_hash and hash in @dev_hash")
    aux["ntags"] = aux["tags"].apply(lambda x: np.sum(x))
    display(aux.query("ntags > 0"))
    # display(aux)

In [ ]:
# drop paragraphs shared between datasets
df_dev.query("hash not in @train_hash", inplace=True)
df_test.query("hash not in @train_hash and hash not in @dev_hash", inplace=True)

In [ ]:
dataset["train"] = Dataset.from_pandas(df_train)
dataset["validation"] = Dataset.from_pandas(df_dev)
dataset["test"] = Dataset.from_pandas(df_test)

dataset = dataset.remove_columns(["__index_level_0__"])

In [ ]:
set(dataset["train"]["hash"]).intersection(set(dataset["validation"]["hash"]))

In [ ]:
dataset

## save dataset

In [ ]:
import srsly

dataset.save_to_disk(f"{OUTPUT_DIR}/hg_dataset")
with open(f"{OUTPUT_DIR}/hg_dataset/label_mapping.json", "w") as file:
    json = srsly.json_dumps(label2code)
    file.write(json)

### export counter of labels

In [ ]:
from collections import Counter

In [ ]:
train_counter = Counter()
for paragraph in dataset["train"]:
    train_counter += Counter(paragraph["tags"])
train_counter = dict(train_counter)
train_counter = {code2label[k]: v for k, v in train_counter.items()}
train_counter = sum(
    map(
        lambda x: Counter({re.sub(r"B-|I-", "", x[0]): x[1]}),
        train_counter.items(),
    ),
    Counter(),
)

val_counter = Counter()
for paragraph in dataset["validation"]:
    val_counter += Counter(paragraph["tags"])
val_counter = dict(val_counter)
val_counter = {code2label[k]: v for k, v in val_counter.items()}
val_counter = sum(
    map(
        lambda x: Counter({re.sub(r"B-|I-", "", x[0]): x[1]}),
        val_counter.items(),
    ),
    Counter(),
)

test_counter = Counter()
for paragraph in dataset["test"]:
    test_counter += Counter(paragraph["tags"])
test_counter = dict(test_counter)
test_counter = {code2label[k]: v for k, v in test_counter.items()}
test_counter = sum(
    map(
        lambda x: Counter({re.sub(r"B-|I-", "", x[0]): x[1]}),
        test_counter.items(),
    ),
    Counter(),
)

In [ ]:
pd.Series(dict(train_counter)).to_csv(f"{OUTPUT_DIR}/train-label-counter.csv")
pd.Series(dict(val_counter)).to_csv(f"{OUTPUT_DIR}/val-label-counter.csv")
pd.Series(dict(test_counter)).to_csv(f"{OUTPUT_DIR}/test-label-counter.csv")

### re export conll files

In [ ]:
from tqdm import tqdm


def hg2conll(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    paragraphs = pd.DataFrame()
    for i, row in tqdm(df.iterrows(), total=len(df)):
        aux = pd.DataFrame(
            {
                "token": row["tokens"],
                "label": [code2label.get(code, "O") for code in row["tags"]],
            }
        )
        last_idx = len(aux)
        aux.loc[last_idx] = np.nan

        paragraphs = pd.concat([paragraphs, aux], ignore_index=True)
    return paragraphs

In [ ]:
train_conll = hg2conll(df_train)
dev_conll = hg2conll(df_dev)
test_conll = hg2conll(df_test)

In [ ]:
train_conll.to_csv(f"{OUTPUT_DIR}/train.txt", sep=" ", header=False, index=False)
dev_conll.to_csv(f"{OUTPUT_DIR}/dev.txt", sep=" ", header=False, index=False)
test_conll.to_csv(f"{OUTPUT_DIR}/test.txt", sep=" ", header=False, index=False)

## TODOs

- [X] Remover de cada resolución los párrafos con etiquetas ambiguas y persistir los csvs individuales. Pensar en cómo hacerlo considerando los índices de inicio de fin de cada párrafo a remover
- [X] Hacer el train - val - test split con proporciones 0.8 - 0.1 - 0.1, que da aproximadamente 428 - 54 - 53 resos.
- [X] Shufflear los párrafos de train.
- [X] Eliminar párrafos duplicados en todos los sets.
- [ ] Entrenar y validar con Flair y con LORA.
- [ ] Hacer downsampling de los párrafos sin etiquetas y repetir.

In [ ]:
to_review = []

for annot in data:
    csv = pd.read_csv(os.path.join(ANNOT_DIR, annot))
    csv["pipes"] = csv["anonymized"].dropna().map(lambda x: True if re.match(r"(?:\w+(?!>)\|\w+\|)+", x) else False)
    if len(csv.loc[(csv["pipes"] == True) & (csv["original"] != csv["anonymized"]) & (csv["label"] == "O")]) > 0:
        to_review.append(annot)

to_review

In [ ]:
to_review

In [ ]:
len(to_review)